# 신경망 학습
**학습** : 훈련 데이터로부터 가중치 매개변수의 최적값   
[손실함수] : 신경망 모델의 예측 값과 실제 값 간의 차이를 측정하는 함수

## 1. 오차 제곱 합
sum of squares for error (SSE)
* 원핫 인코딩을 함.

$$ E = \frac{1}{2} \displaystyle\sum_k(y_k-t_k)^2$$

In [1]:
def sum_squares_error(y,t):
    return 0.5*np.sum((y-t)**2)

## 2. Cross Entropy Error(CEE)   
교차 엔트로피 오차

$$E = -\displaystyle\sum_k\;t_k\;\text{log}y_k$$

In [2]:
def cross_entropy_error(y,t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))

delta를 더하는 이유는 np.log()함수엥 0을 입력하면 마이너스 무한대.
따라서 아주 작은 값을 더해서 0이 될 수 없게 했음.

## 3. 미니배치
$$ E= -\frac {1}{N} \displaystyle\sum_n\displaystyle\sum_k\; t_{nk}\; \text{log}\;y_{nk} $$

일부만 골라서 학습을 수행하는것.  
**미니배치**


In [ ]:
import sys, os
sys.path.append(os.path.join(os.path.dirname(__file__),'..'))
import numpy as np
from dataset.mnist import loasd_mnist

(x_train,t_train),(x_test, t_test) = load_mnist(normalize=True, one_hot_label = True)

print(x_train.shape) # (60000,784)
print(t_train.shape) # (60000,10)

호출할 때 one_hot_label=True로 지정하여 원핫인코딩으로,   
앞의 코드에서 MNIST 데이터를 읽은 결과,    
훈련데이터는 60,000개고 입력 데이터는 784열 (원래는 28x28)인 이미지 데이터임을 할 수 있다.     
정답레이블은 10줄짜리 데이터임.

### 왜 손실 함수를 설정하는가?
신경망을 학습할 때 정확도를 지표로 삼아서는 안 된다.  
정확도를 지표로 하면 매개변수의 미분이 대부분의 장소에서 0이 되기 때문이다.

## 4. 미분
해석적미분   
$\frac{df(x)}{dx}=\displaystyle\lim_{h\rarr\infin}{\frac{f(x+h)-f(x)}{h}}$
-> 반올림 오차 문제 일으킴   
수치미분 numerical differentiation   
**중심차분** **중앙차분**으로 해결   
$\frac{df(x)}{dx}=\displaystyle\lim_{h\rarr\infin}{\frac{f(x+h)-f(x-h)}{2h}}$

In [4]:
def numerical_diff(f,x):
    h=1e-4 # 0.0001
    return(f(x+h)-f(x-h)/(2*h))

### 편미분

$f(x_0,x_1)\;=\;x^2_0+x^2_1$

In [ ]:
def function_2(x):
    return x[0]**2+x[1]**2
#또는 return np.sum(x**2)

문제: $x_0 = 3, x_1 = 4$ 일때, $x_0$ 에 대한 편미분 $\frac{\partial{f}}{\partial{x_0}}$ 를 구하라.

In [7]:
def function_tmp1(x0):
    return x0*x0 +4.0**2**0

numerical_diff(function_tmp1,3.0)

-64983.99944998999

## 5. 기울기

기울기가 가르키는 쪽은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향

In [8]:
def numerical_gradient(f,x):
    h = 1e-4 #0.0001
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성
    
    for idx in range(x.size):
        tmp_val = x[idx]
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val #값 복원
    return grad

In [ ]:
#학습률
def gradient_descent(f,init_x,lr=0.01,step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f,x)
        x -= lr*grad
    return x
# 인수 f는 최적화 하려는 함수, init_x는 초깃값, lr은 learning rate를 의미. step_num은 경사법에 따른 반복 횟수
# 학습률을 곱한 값으로 갱신하는 처리를 step_num번 반복


신경망에서도 기울기를 구해야함.   
가중치 매개변수에 대한 손실함수에 대한 기울기.
### simpleNet

In [ ]:
import numpu as np
import sys,os
sys.path.append(os.path.join(os.path.dirname(__file__),'..'))
from common.function import softmax, cross_entropy_error
from common.gradient import numerical_gradient
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) #정규분포로 초기화
        
    def predict(self,x):
        return np.dot(x,self.W)
    def loss(self,x,t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y,t)
        return loss

common/functions.py에 정의한 softmax와 cross_entropy_error 매서드를 의용함